## $\S$2. CNN

### 2.1 CNN网络简介

#### 一. 计算机视觉相关问题
1. 用于计算机视觉的神经网络, 其每层节点的作用不同: 如  
  1. 第一层网络检测物体边界
  2. 后面的网络检测出物体
  3. 最后面的网络检测出完整的物体, 如一张完整的人脸

2. 如下一张灰度图, 我们要检测出他的垂直边界.   
  1. 灰度图使用6\*6\*1的矩阵表示 (因为不是rgb图像, 所以不是6\*6\*3)  
  2. 我们定义一个3\*3的"过滤器"矩阵filter, 该矩阵也称为"核"  
  3. 使用灰度值矩阵和filter进行卷积操作  
   中间的星号代表卷积操作 : 将filter放在灰度值矩阵的顶端, 把相对应的元素进行乘积求和, 作为结果矩阵的第一个元素.   
   然后移动filter一列, 和对应的弧度制矩阵做相同的乘积求和(卷积)操作, 结果作为第二个元素. 以此类推 ..   
       <img src="../img/edgedetection">  
3. 卷积操作的编程实现  
  1. tensorflow中 : `tf.nn.conv2d`
  2. keras中 : `Conv2D`
  
4. 为了阐述卷积操作如何进行边沿检测, 我们使用一个简单的例子<span id="edgedetection1"></span>  
  1. 使用左侧全为10, 右侧全为0的灰度值矩阵, filter使用3\*3的矩阵  
  2. 若0表示灰色, 正值表示白色, 负值表示黑色. 则卷积操作的结果为"中间白色, 两边灰色的图像". 中间的白色条即为检测出的垂直边界
  <img src="../img/edgedetection2.png" height="60%" width="60%">
  
#### 二. 更多的边缘检测
<a href="#edgedetection1">上面第4点中</a>, 介绍了手动设置边缘过滤矩阵filter, 现在来学习, 算法如何自动识别边缘
1. 其他的2个过滤器选择  
 上面垂直边缘检测的例子中, filter矩阵为$\begin{bmatrix} 1 & 0 & -1 \\ 1 & 0 & -1 \\ 1 & 0 & -1 \end{bmatrix}$. 实际上有很多种不同的一致filter,例如  
   1. sobel filter : $\begin{bmatrix} 1 & 0 & -1 \\ 2 & 0 & -2 \\ 1 & 0 & -1 \end{bmatrix}$  
   2. scharr filter : $\begin{bmatrix} 3 & 0 & -3 \\ 10 & 0 & -10 \\ 3 & 0 & -3 \end{bmatrix}$
   
2. 事实上, filter的9个参数, 是通过反向传播学习而来. 具体方法后续介绍, 现在来看2个重要概念

### 2.2 卷积运算的两个变形

#### 一. Padding (填补)
1. 卷积操作有两个特点:   
 一是原始图像在和filter矩阵卷积后, 输出矩阵会变小. 例如: 若原始图像为6\*6,filter为3\*3,则最终结果矩阵为4\*4  
 二是, 原始矩阵中, 中间块的灰度值会被多个3\*3子矩阵交叉, 而边缘像素只能被1个3\*3矩阵包围, 导致卷积操作时, 边缘像素点的关注度很小
 
2. 为了解决上述问题, 出现了padding操作  
 即在原始图像的四周增加一圈灰度值=0的像素点, 使得结果矩阵变大, 同时使得原来处于边缘的像素点, 在扩大输入图像后, 处于中间位置.  
 例如 : 把6\*6图像在padding=1后, 形成8\*8的输入图像. 此时结果变成6\*6的矩阵 (和原始图像大小一样), 同时照顾了原始图像的边缘像素点
<img src="../img/padding1.png" height='80%' width='80%'>
3. 如何选取padding的大小  
 有两种选择padding大小的方法 :  
  1. "Valid Padding": 即p=0, 不在图像周围填充任何像素.
  2. "Same Padding" : 填充p后, 使得输出图像大输入图像的大小相同  
   即: 若输入图像为$n*n$, filter为$f*f$, 为了让输出矩阵也为$n*n$, 则有$(n+2p)-f+1=n$, 则 $p=\frac { f-1 }{ 2 } $
   
#### 二. Stride convolution
1. stride convolution改变了每次进行卷积后移动的步长.(传统卷积运算后只移动一个像素)  
 <img src="../img/strdieconvolution1.png" height='60%' width='60%'>
 若stride=s,则输出图像像素宽度和长度为$\left\lfloor \frac { n-f }{ s } +1 \right\rfloor $  
 $\left\lfloor z \right\rfloor $ = floor(z), 意为向下取整  
 如上图, 输出图像长宽为$\left\lfloor \frac { 7-3 }{ 2 } +1 \right\rfloor =3$
 
#### 三. 总结  
 若同时进行padding和stride, 且stride=s,padding=p. 则输出图像像素宽度和长度为$\left\lfloor \frac { n+2p-f }{ s } +1 \right\rfloor $  
 
#### 四. cross-correlation与convolution
1. 数学上的卷积操作, 比这里的卷积操作多了一步: 即在进行卷积前, 先把filter矩阵在横轴和纵轴翻转一下, 然后在和输入矩阵进行卷积  
2. 但是深度学习中, 常常省略filter的翻转操作, 并仍称之为convolution运算. 数学上把这个简化的过程称之为croos-correlation

#### 五. RGB三维通道上的卷积运算
1. 取代灰度值作为输入矩阵, 则输入矩阵变成堆叠在一起的3个正方形. 每个正方形代表图像上每个像素点的R/G/B通道值.
2. `filter`矩阵也变成三维的.如图`3*3*3`的filter矩阵在进行convolution运算时, 要进行27次相乘再求和.  
 `filter`矩阵的每一个正方形, 分别来检测R/G/B某个通道上的边缘. 如只检测R通道的边缘, 则是用其他通道维度上对应的filter正方形全0.  
 只检测R通道的边缘的`filter`矩阵 : $\quad \quad R\quad \quad \quad \quad \quad G\quad \quad \quad \quad \quad B\\ \begin{bmatrix} 1 & 0 & -1 \\ 1 & 0 & -1 \\ 1 & 0 & -1 \end{bmatrix}\begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 0 \end{bmatrix}\begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 0 \end{bmatrix}$, 此时输出的是`4*4`的2维矩阵   
3. 神经网络中, 有时想同时检测出垂直边缘和水平边缘. 此时应该让输入矩阵同时卷积2个`filter`矩阵.此时的输出矩阵也是三维的    
 <img src='../img/convolutionvolumn.png' height='70%' width='70%'>

### 2.3 卷积神经网络的构建

#### 一. 单层CNN构建
1. 单层卷积网络的正向传播:   
 输入RGB图像, 用2个filter矩阵进行卷积, 输出`4*4*2`矩阵   
 CNN网络中, `filter矩阵`的作用是产生feature, n个`filter矩阵`产生n个frature. 同时, `filter矩阵`个数也是输出矩阵的`#channels数/深度` 
 <img src='../img/convolution2.png' height='80%' width='80%'>
2. 单层CNN的符号总结
<img src='../img/1layercnn.png'> 

#### 二. 深度CNN网络
1. ConvNet: 由多个卷积层形成的神经网络   
 3层卷积层 + 1层softmax层
<img src="../img/cvnnet1.png">

#### 三. Pooling运算
1. Max pooling运算 - 最大值pooling  
 计算pool里数值中的最大值  (filter矩阵的移动同卷积操作)   
 <img src='../img/maxpolling1.png' width='50%' height='50%'>
2. 均值Pooling   
 计算pool里数值的均值 (filter矩阵的移动同卷积操作)
 <img src='../img/polling2.png' width='50%' height='50%'>
3. Pooling的hyper parameter   
 只需人为设定或交叉验证filter大小/stride值


### 2.4 神经网络举例

#### 一. LeNet-5网络举例
1. 目标 : 识别图像中的数字
2. 网络结构  
  1. 输入图像`32*32*3`
  2. layer1=卷积层1+池化层1: 最终输出`14*14*6`  
   layer2=卷基层2+池化层2: 最终输出`5*5*16`  
  3. 展开layer2的输出为$R^{400}$的向量, 经过全连接层3网络输出$R^{120}$
  4. 经过全连接层4网络输出$R^{84}$
  5. softmax输出10个数字的概率  
<img src='../img/letnet5.png' width='80%' height='80%'>  
[注1]: 当神经网络的深度更深时, 输出矩阵的宽度和高度会减小,而#channels数量会增多  
[注2]: 因为神经网络在计算层数时, 只把那些有权重w/偏差项b的算作一层. 而pooling操作没有参数(只有超参数), 因此有时并不把pooling层单独算作神经网络的一层

#### 二. 神经网络为何在计算机视觉如此有用
1. Parameter sharing - 参数共享 :  
 传统的全连接网络, 输入图像矩阵的每个像素点都要有一个参数w与之计算. 而卷积操作的fiter矩阵, 其中的元素值实际上共享的参数. 因为fiter矩阵在图像的这部分作为参数进行卷积, 也在图像的其他部分进行卷积  
2. Sparsity of connetions - 稀疏连接 : 
 每一层的输出矩阵中的元素值, 都依赖于输入图像的一小部分
 
#### 三. 识别猫的例子
1. 人工标注正负例
2. 卷积网络 - softmax分类输出
2. 梯度下降优化最优解  

<img src='../img/recognisecat1.png' width='70%' height='70%'>